In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from tensorflow.keras.models import Sequential      #This allows appending layers to existing models
from tensorflow.keras.layers import Dense           #This allows defining the characteristics of a particular layer
from tensorflow.keras import optimizers             #This allows using whichever optimiser we want (sgd,adam,RMSprop)
tf.keras.backend.set_floatx('float64')

def loss(model, x, t):
    with tf.GradientTape() as tape_t:
        tape_t.watch([t])

        dx_dt = tape_t.gradient(x, t)
    #rhs1 = -x + (tf.transpose(x) @ x @ A + (1 - tf.transpose(x) @ A @ x) @ np.eye(6)) @ x
    
    for i in range(Nt):
        x_i = x[6 * i : 6* i + 6]
        
    rhs1 = -x
    rhs2 = x @ tf.transpose(x) @ A
    rhs3 = (1 - x @ A @ tf.transpose(x))
    rhs = rhs1 + (rhs2 + rhs3) @ x

    return tf.losses.mean_squared_error(zeros, dx_dt - rhs)

def train(model, optimizer, x, t):
    with tf.GradientTape() as tape:
        current_loss = loss(model, x, t)
        
    grads = tape.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

# Setting up data
Q = np.random.rand(6,6)
A = (Q.T + Q)/2

Nt = 2
x_np = np.random.rand(6,1)
t_np = np.linspace(0, 1, Nt)

X, T = np.meshgrid(x_np, t_np)

x = X.ravel()
t = T.ravel()

zeros = tf.reshape(tf.convert_to_tensor(np.zeros(x.shape)), shape=(-1,1))
x = tf.reshape(tf.convert_to_tensor(x), shape=(-1,1))
t = tf.reshape(tf.convert_to_tensor(t), shape=(-1,1))

# Setting up model
model = Sequential()
model.add(Dense(20, activation='sigmoid'))
model.add(Dense(1, activation="linear"))
model.build(tf.concat([x,t], 1).shape)

eta = 0.001
sgd = optimizers.SGD(lr=eta)

# Training model
num_iter = 100000
for i in range(num_iter):
    train(model, sgd, x, t)
    
# Output of model
g_dnn = (1 - t) * tf.sin(np.pi * x) + x * (1 - x) * t * model(tf.concat([x,t], 1))
G_dnn = np.array(g_dnn).reshape((Nt, Nx))

